## MNIST(kaggleバージョン)をPytorchで実行する。
### やること
1. データの読み込み〜Dataset化
2. Dataloaderの定義
3. ネットワークの定義
4. 学習処理の定義
5. 検証用データを用いたモデルの検証
6. テストデータへの適用

### 参考
- Dataset, DataLoader：  
    - https://www.kaggle.com/pinocookie/pytorch-dataset-and-dataloader  
    - https://qiita.com/mathlive/items/2a512831878b8018db02
- 学習処理の記述方法：
    - https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py
- データランダムな分割の方法：
    - https://qiita.com/takurooo/items/ba8c509eaab080e2752c
- Epochごとのモデルの検証：
    - https://axa.biopapyrus.jp/deep-learning/pytorch/image-classification.html

## ライブラリの読み込み

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
import pandas as pd
from tqdm import tqdm

## モデルの定義

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__() ## run the Parent Class's .__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, padding=1) # (in_channnel, out_channel, kernel_size)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, padding=1)
        self.conv2_drop = nn.Dropout2d(p=0.6)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(784, 240)  # paddingすることで画像サイズは据え置きになるため784とする。
        self.fc2 = nn.Linear(240, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fc4 = nn.Softmax()

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=(2, 2)) ## J
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2_drop(self.conv2(x))), (2, 2))
        x = x.view(-1, self.num_flat_features(x)) # x.shape = (8,400)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.fc4(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net().double()
print(net)

## Transformの定義(このNotebookでは使用しない。勉強のための定義のみ。)
- ここでは、データを格納しているTensorを元にDatasetクラスを作成し、Pytorch上で扱いやすくする。
- TorchVision標準のデータセットを用いる場合、以下の流れとなるらしい。(今回は不使用。)
    - transformsによる前処理の定義(torchvision)
        - transform：データの前処理全般を定義するもの。
        - 今回は各pixelの値を255.0で除算する前処理を入れる。
        - 画像なら(縦、横、チャネル)->(チャネル、縦、横)と変換する、ピクセル数を補正する、左右反転、回転させるなどの機能がある。
    - Datasetsによる前処理&ダウンロード
    - DataloaderによるDatasetの使用

In [ ]:
class ImageTransform():
    """
    画像に対する前処理を行うクラス。以下の機能を持つ。
    pandas.DataFrameを入力としてTensorに変換する。
    その後、各pixelの値を0〜1にするため、255で除算する。
    なお、学習時と診断時での処理内容は同じであるとする。
    
    Attributes
    ----------
    None
    
    """
    def __init__(self):
        pass
    
    def __call__(self, df_pic):
        """
        学習時と診断時での前処理は同じものとする。
        本当は学習時のみノイズを加えたり色々する。
        """
        out_label = df_pic.label
        out_img = df_pic.iloc[:,1:].values.astype(np.uint8).reshape((1, 28, 28))
        out_label = torch.tensor(y_train.values)
        out_img = torch.tensor(x_train.values)
        out_img = out_img/255.0
        return out_img, out_label

trans = ImageTransform

## DataSetsの作成
- torch.utils.data.Datasetクラスを継承して作成する。
- メソッドには以下がある。
    - init:コンストラクタ、クラス定義時に実行され、値の初期化などを行う。
    - len:画像の枚数を返す。
    - getitem:Datasetからデータを1レコード分返す。DataLoaderに呼び出されている？

In [ ]:
class MnistDatasets(data.Dataset):
    def __init__(self, input_filename, transform = None):
        self.data = pd.read_csv(input_filename)
        self.transform = trans
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        if self.data.shape[1]==784:
            img_transformed = self.data.iloc[idx,:].values.astype(np.uint8).reshape((1, 28, 28))/255.0
            return img_transformed
        else:
            img_transformed = self.data.iloc[idx, 1:].values.astype(np.uint8).reshape((1, 28, 28))/255.0
            img_label = self.data.iloc[idx, 0]
            return img_transformed, img_label


## 教師データの学習用データと検証用データへの分割

In [ ]:
data_supervised = MnistDatasets("./01_input/train.csv")
n_eval = 2000
data_train, data_eval = data.random_split(data_supervised, [42000-n_eval, n_eval])
data_test = MnistDatasets("./01_input/test.csv")

## Dataloaderを作成する

In [ ]:
n_minibatch = 8
trainloader = data.DataLoader(data_train, batch_size=n_minibatch, shuffle=False)
evalloader = data.DataLoader(data_eval, batch_size=n_minibatch, shuffle=False)
testloader = data.DataLoader(data_test, batch_size=n_minibatch, shuffle=False)

## 損失関数と最適化手法を定義する
損失関数には交差エントロピー損失を使用する。

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), weight_decay=0.0005)
# optimizer = optim.SGD(net.parameters(), lr=0.0005, momentum=0.9, weight_decay=0.0001)

## これまでに作成した関数を用いて学習を行なう。
- パラメータの初期化
- for epoch
    - for ミニバッチ学習
        - 順伝播
        - 損失の計算
        - パラメータ最適化
            - 損失の勾配の計算
            - パラメータの更新
    - for 検証用データでテスト
        - 順伝播
        - 損失の計算
- todo：学習/検証の処理フェーズを条件分岐で記述する。⇒TBD

In [ ]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    net.train()
    for i, data_jj in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data_jj
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.double())
        train_loss = criterion(outputs, labels)
        train_loss.backward()
        optimizer.step()

        # print statistics
        running_loss += train_loss.item()
        if i % 5000 == 4999:    # print every 5000 mini-batches
            print('[%d, %5d] Training Loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5000))
            running_loss = 0.0
    print("Epoch No." + str(epoch+1) + " has finished!")
    
    eval_loss = 0
    net.eval()
    for i, data_jj in enumerate(evalloader, 0):
        inputs, labels = data_jj
        outputs = net(inputs.double())
        eval_loss = criterion(outputs, labels)
        
        # print statistics
        running_loss += eval_loss.item()
    
    print('Validation Loss: %.3f' %
          (running_loss/(n_eval/n_minibatch))) ## eval_lossを追加
    print('*****')

print('Finished Training')

In [ ]:
PATH = './02_output/MNISTmodel.pth'
torch.save(net.state_dict(), PATH)

## 学習済みモデルを用いた予測の実行
- メモリにデータが乗り切るよう、100件ごとに予測値を算出する。
- PCのメモリが足りず全データを一度に処理できなかったため。

In [ ]:
a = 0
for i, data_jj in enumerate(testloader, 0):
    inputs = data_jj
    tmp_pred = net(inputs.double())
    tmp_pred = np.argmax(tmp_pred.detach().numpy().copy(), axis=1)
    if a==0:
        pred = tmp_pred
        a+=1
    else:
        pred = np.r_[pred, tmp_pred]

output = pd.DataFrame(pred)
output.columns = ["Label"]
output.index.name = "ImageID"
output.index = output.index + 1

output.to_csv("./02_output/pred_labels.csv")

## その他：Pythonにおけるクラスの概念
- "class hogehoge():" として定義する。引数には継承元の親クラスが入る。
- アンダースコア2本で囲まれるメソッドは特殊メソッドと呼ばれる。
    - init：インスタンスを生成する時点で実行される。
    - call：生成したインスタンスを()つきで実行すると実行される。
    - getitem:生成したインスタンスを[]つきで実行すると実行される。鍵カッコにはインデックスを示す整数が入る。
- self.super().｛メソッド名｝とすると親クラスのメソッドを利用できる。